In [1]:
# Calculate Upper Limits for Each source in the unblinded source list and format as a latex table

In [2]:
import csky as cy
import pandas as pd
import numpy as np

In [3]:
import sys
sys.path.insert(0, '../..')

import config as cg
import trials as trials
cg.base_dir = '/data/user/mhuennefeld/data/analyses/unblinding_v1.0.0/'

Running as User: ssclafani on Hostname: cobalt05.icecube.wisc.edu


In [4]:
base_dir = '/data/user/ssclafani/data/analyses/baseline_unblind_v1.0.0/'

In [5]:
results = np.load(base_dir + 'ps/results/fulllist_unblinded.npy', 
                  allow_pickle=True)

In [6]:
source_list = pd.read_pickle('../../catalogs/Source_List_DNNC.pickle')

In [7]:
RA = source_list.to_numpy()[:,1]
DEC = source_list.to_numpy()[:,2]

In [8]:
repo = cy.selections.Repository(local_root='/data/user/ssclafani/data/analyses')
specs = cy.selections.DNNCascadeDataSpecs.DNNC_10yr

In [9]:
selection_version = 'version-001-p00'

In [10]:
%%time

ana = cy.get_analysis(
    repo, selection_version, specs, 
    #gammas=np.r_[0.1:6.01:0.125],
)

Setting up Analysis for:
DNNCascade_10yr
Setting up DNNCascade_10yr...
Reading /data/user/ssclafani/data/analyses/dnn_cascades/version-001-p00/MC_NuGen_bfrv1_2153x.npy ...
Reading /data/user/ssclafani/data/analyses/dnn_cascades/version-001-p00/IC86_2011_exp.npy ...
Reading /data/user/ssclafani/data/analyses/dnn_cascades/version-001-p00/IC86_2012_exp.npy ...
Reading /data/user/ssclafani/data/analyses/dnn_cascades/version-001-p00/IC86_2013_exp.npy ...
Reading /data/user/ssclafani/data/analyses/dnn_cascades/version-001-p00/IC86_2014_exp.npy ...
Reading /data/user/ssclafani/data/analyses/dnn_cascades/version-001-p00/IC86_2015_exp.npy ...
Reading /data/user/ssclafani/data/analyses/dnn_cascades/version-001-p00/IC86_2016_exp.npy ...
Reading /data/user/ssclafani/data/analyses/dnn_cascades/version-001-p00/IC86_2017_exp.npy ...
Reading /data/user/ssclafani/data/analyses/dnn_cascades/version-001-p00/IC86_2018_exp.npy ...
Reading /data/user/ssclafani/data/analyses/dnn_cascades/version-001-p00/IC86

In [12]:
%%time
tss = []
pre_trial_p = []
nss = []
gammas = [] 
ul90_ns = []
ul90_flux = []

for i, result in enumerate(results[0]):
    if i % 5 == 0:
        print(i)
    #src = cy.utils.sources(ra=np.deg2rad(float(RA[i])), dec=np.deg2rad(float(DEC[i])))
    #conf = cg.get_ps_conf(src, gamma=2.0)
    #tr = cy.get_trial_runner(ana = ana, conf=conf)
    #name = 'ts_{:04}'.format(i)
    #ul = tr.find_n_sig(ts=result[0], beta=0.9, tol=.02, 
    #                   batch_size=1000, mp_cpus=18, n_sig_step=10, logging=True)
    #ul90_ns.append(ul['n_sig'])
    #ul90_flux.append(tr.to_E2dNdE(ul['n_sig'], E0=100, unit=1e3))
    tss.append(result[0])
    nss.append(result[1])
    gammas.append(result[2])
    pre_trial_p.append(result[3])
    
    

0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
CPU times: user 930 µs, sys: 1.01 ms, total: 1.94 ms
Wall time: 1.39 ms


In [44]:
#names = source_list['Names']

In [45]:
#names = names.to_numpy()

In [46]:
names = source_list.SRC.to_numpy()

In [48]:
print(r'Source Number & Source Name & RA & Dec & TS & ns & gamma & pvalue$_{pre}$')
for i, ts in enumerate(tss):
    if ts == 0:
        print('{} & {} & {:.1f} & {:.1f} & {:.1f}  & -- & -- & -- \\\\'.format(i, names[i], RA[i], DEC[i], tss[i], nss[i], gammas[i], pre_trial_p[i]))
    else:
        print('{} & {} & {:.1f} & {:.1f} & {:.1f}  & {:.1f} & {:.1f}  & {:.4f} \\\\'.format(i, names[i], RA[i], DEC[i], tss[i], nss[i], gammas[i], pre_trial_p[i]))
    

Source Number & Source Name & RA & Dec & TS & ns & gamma & pvalue$_{pre}$
0 & G343.1-2.3 & 257.0 & -44.3 & 8.9  & 77.5 & 2.9  & 0.0110 \\
1 & HESS J0835-455 & 128.3 & -45.2 & 0.0  & 0.2 & 2.0  & 0.7190 \\
2 & PKS 0426-380 & 67.2 & -37.9 & 0.1  & 3.0 & 2.3  & 0.6404 \\
3 & PKS 2155-304 & 329.7 & -30.2 & 1.7  & 61.3 & 4.0  & 0.2504 \\
4 & Mkn 421 & 166.1 & 38.2 & 0.0  & -- & -- & -- \\
5 & PKS 0537-441 & 84.7 & -44.1 & 0.2  & 1.1 & 1.8  & 0.5762 \\
6 & PKS 0447-439 & 72.4 & -43.8 & 0.0  & 0.7 & 2.1  & 0.6979 \\
7 & BL Lac & 330.7 & 42.3 & 0.8  & 7.6 & 1.6  & 0.3398 \\
8 & PG 1553+113 & 238.9 & 11.2 & 0.0  & -- & -- & -- \\
9 & TXS 0518+211 & 80.4 & 21.2 & 5.3  & 149.3 & 3.4  & 0.0355 \\
10 & PKS 0235+164 & 39.7 & 16.6 & 0.0  & -- & -- & -- \\
11 & PKS 1424+240 & 216.8 & 23.8 & 3.4  & 7.7 & 1.3  & 0.0944 \\
12 & 3C 66A & 35.7 & 43.0 & 1.5  & 86.3 & 3.7  & 0.2366 \\
13 & TXS 0506+056 & 77.4 & 5.7 & 0.0  & -- & -- & -- \\
14 & AP Librae & 229.4 & -24.4 & 0.0  & 0.3 & 1.0  & 0.7247 \\
15 & S